In [2]:
!pip install scikit-learn

In [16]:
!pip install breakpoint

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\jean-\anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\jean-\\AppData\\Local\\Temp\\pip-install-9huq9dv1\\breakpoint_915b04109be24598a3407d29f63278bc\\setup.py'"'"'; __file__='"'"'C:\\Users\\jean-\\AppData\\Local\\Temp\\pip-install-9huq9dv1\\breakpoint_915b04109be24598a3407d29f63278bc\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\jean-\AppData\Local\Temp\pip-pip-egg-info-cnp20t7o'
         cwd: C:\Users\jean-\AppData\Local\Temp\pip-install-9huq9dv1\breakpoint_915b04109be24598a3407d29f63278bc\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      F

In [8]:
from pygam import LinearGAM, s, f
from breakpoint import breakpoint
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [9]:
nb_actifs = 20
nb_dates = 256 * 5

idx = pd.Index(np.arange(nb_actifs))

mu = pd.Series(0.05,index=idx)
vols = pd.Series(0.2,index=idx)
correl = 0.7

covar = np.diag(vols) @ pd.DataFrame(correl * np.ones((nb_actifs,nb_actifs)) + (1-correl) * np.eye(nb_actifs),index=idx,columns=idx) @ np.diag(vols)
A = np.linalg.cholesky(covar)

rdts_observes = mu/256 + pd.DataFrame(np.random.randn(nb_dates,nb_actifs)) @ A.T / 16


In [13]:
X=rdts_observes
Y=np.ones(nb_dates)

[1. 1. 1. ... 1. 1. 1.]


In [14]:
best_score = float('-inf')
best_num_splines = None

for num_splines in range(1, 11):  
    gam = LinearGAM(s(0, n_splines=num_splines))
    scores = cross_val_score(gam, X.reshape(-1, 1), Y, cv=5, scoring='r2')
    avg_score = np.mean(scores)

    if avg_score > best_score:
        best_score = avg_score
        best_num_splines = num_splines

print("Nombre optimal de splines:", best_num_splines)

# Utilisation du nombre optimal pour ajuster une régression segmentée
segmented_gam = LinearGAM(s(0, n_splines=best_num_splines)).fit(X.reshape(-1, 1), Y)

# Tracé des données et de la régression segmentée
import matplotlib.pyplot as plt

plt.scatter(X, Y, label='Données')
plt.plot(X, segmented_gam.predict(X.reshape(-1, 1)), label='Régression Segmentée', color='red')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

ValueError: n_splines must be > spline_order. found: n_splines = 1 and spline_order = 3